In [51]:
import pandas as pd
import numpy as np
import os, sys, pytz
import matplotlib.pyplot as plt
from tqdm import tqdm
from warnings import filterwarnings
filterwarnings('ignore')

import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from sklearn.metrics import r2_score
from scipy.stats import spearmanr
import wandb
import random
import pickle

import pandas as pd
import numpy as np
import alpaca
from alpaca_trade_api import REST
import matplotlib.pyplot as plt
import datetime
from tqdm import tqdm
import pytz

### get stock data
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.data.historical import StockHistoricalDataClient

import sys
sys.path = sys.path + ['../']

import my_secrets
import pickle

from warnings import filterwarnings
filterwarnings('ignore')

import time

In [52]:
date_start = datetime.datetime.strptime('2018-01-01-00:00:00','%Y-%m-%d-%H:%M:%S')
date_end = datetime.datetime.strptime('2023-04-02-00:00:00','%Y-%m-%d-%H:%M:%S')

In [53]:
stock_code='TQQQ'

In [4]:
import os
# if os.path.exists(f'./historical_price_data/{stock_code}.csv'):
#     continue
request_params = StockBarsRequest(
    symbol_or_symbols=stock_code,
    timeframe=TimeFrame.Minute,
    start=date_start,
    end=date_end, 
    adjustment='all'
)

client = StockHistoricalDataClient(api_key=my_secrets.API_KEY, secret_key=my_secrets.SECRET_KEY) ### request stock price
bars = client.get_stock_bars(request_params)
bas_df = bars.df.droplevel(axis=0,level=0)
bas_df.index = [i.replace(tzinfo=None).tz_localize(pytz.timezone('UTC')).tz_convert('US/Eastern') for i in bas_df.index]
bas_df.to_csv(f'./historical_price_data/{stock_code}.csv',index=True)




KeyboardInterrupt



In [54]:


path = f'./historical_price_data/{stock_code}.csv'
data = pd.read_csv(path,index_col=0)
data.index = pd.Series([i.replace(tzinfo=None) for i in pd.to_datetime(data.index)]).dt.tz_localize(pytz.timezone('US/Eastern'))

a = data.resample('1min').mean().dropna()
a['change_rate'] = (a['close'] - a['open'])/a['open']
a['change_bi'] = np.where((a['close'] - a['open'])>0,1,0)
a['date'] = pd.to_datetime([i.date() for i in a.index])
a['year'] = a.date.dt.year
a['WOY'] = a.date.dt.week
a['WOY_year'] = [str(a)+str(b) for a,b in zip(a['WOY'],a['year'])]
a['MOY'] = a.date.dt.month
a['MOY_year'] = [str(a)+str(b) for a,b in zip(a['MOY'],a['year'])]
a['DOW'] = a.date.dt.day_of_week
a['DOY'] = a.date.dt.day_of_year
a['DOY_year'] = [str(a)+str(b) for a,b in zip(a['DOY'],a['year'])]
a['MOD'] = [i.hour*60+i.minute for i in a.index]
a['HOD'] = [i.hour for i in a.index]
a = a[(a.MOD>=9.5*60) & (a.MOD<=16*60)]
def calc_one_day(df):
    start = df.iloc[0,:]['close']
    df['change_from_today_start'] = (df['close'] - start)/start
    return df
a = a.groupby('date').apply(calc_one_day)
a.groupby('MOD').mean()['change_from_today_start'].plot()
#     plt.title(code)
#     plt.show()
a['code'] = stock_code




KeyboardInterrupt



In [ ]:
a = a.merge(
    a[['DOY_year','change_rate']].groupby('DOY_year').sum().reset_index(drop=False).rename(columns={'change_rate':'change_rate_DOY_year'}),
    left_on='DOY_year', right_on='DOY_year',how='left'
).merge(
    a[['WOY_year','change_rate']].groupby('WOY_year').sum().reset_index(drop=False).rename(columns={'change_rate':'change_rate_WOY_year'}),
    left_on='WOY_year', right_on='WOY_year',how='left'
).merge(
    a[['MOY_year','change_rate']].groupby('MOY_year').sum().reset_index(drop=False).rename(columns={'change_rate':'change_rate_MOY_year'}),
    left_on='MOY_year', right_on='MOY_year',how='left'
)



In [ ]:
%load_ext autoreload
%autoreload 2

In [70]:
((6.5*60)/1) * 365

142350.0

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wand

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


In [ ]:
### predict based on past 15 days. 5min interval, 2h+30min+4h = 6.5h. ((6.5*60)/1) * 15 = 5850 block
# each interval being predicted using the past 1170 block

### predict based on past 15 weeks. 5min interval, 2h+30min+4h = 6.5h. ((6.5*60)/1) * 15 * 7 = 40950 block

### predict based on past 15 month. 5min interval, 2h+30min+4h = 6.5h. ((6.5*60)/1) * 15 * 30= 175500 block

feature_dim_minute = 5850
feature_dim_day = 30
feature_dim_week = 15
feature_dim_month = 15

mi_value = [i for i in a['change_rate'].values]
day_value = [i for i in a['change_rate_DOY_year'].values]
uniq_day_value = list(a['change_rate_DOY_year'].unique())
week_value = [i for i in a['change_rate_WOY_year'].values]
uniq_week_value = list(a['change_rate_WOY_year'].unique())
month_value = [i for i in a['change_rate_MOY_year'].values]
uniq_month_value = list(a['change_rate_MOY_year'].unique())

df = []
batch_count = 0
for index,v in tqdm(enumerate(zip(mi_value,day_value,week_value,month_value)), total=len(mi_value)):
    if index<feature_dim_month*(6.5*60)*22+10:
        continue
    
    ### month level, 15 month
    this_month_index = uniq_month_value.index(month_value[index])
    month_list = uniq_month_value[int(this_month_index-feature_dim_month):this_month_index][::-1] ### arround 25 days per month for trading
    
    ### week level, 15 week
    this_week_index = uniq_week_value.index(week_value[index])
    week_list = uniq_week_value[int(this_week_index-feature_dim_week):this_week_index][::-1] ### arround 5 days per week for trading
    
    ### day level, 30 days
    this_day_index = uniq_day_value.index(day_value[index])
    day_list = uniq_day_value[int(this_day_index-feature_dim_day):this_day_index][::-1]

    ### minitue level, 5850 minutes
    minute_list = mi_value[int(index-feature_dim_minute):index][::-1]
    
    df.append({
        **{'y':v[0]},
        **{f'm{i}':j for i,j in zip(list(range(feature_dim_month)),month_list)},
        **{f'w{i}':j for i,j in zip(list(range(feature_dim_week)),week_list)},
        **{f'd{i}':j for i,j in zip(list(range(feature_dim_day)),day_list)},
        **{f'mi{i}':j for i,j in zip(list(range(feature_dim_minute)),minute_list)}
    })
    
    if len(df)>=5120:
        df = pd.DataFrame(df)
        with open(f'./training_data/{stock_code}_processed_data_{batch_count}.pkl','wb') as f:
            pickle.dump(df,f)
        batch_count+=1
        df=[]

        


In [13]:
df

,y,m0,m1,m2,m3,m4,m5,m6,m7,m8,...,mi5840,mi5841,mi5842,mi5843,mi5844,mi5845,mi5846,mi5847,mi5848,mi5849
0,0.000000,0.018896,0.068262,0.124534,-0.350429,-0.026531,-0.398792,-0.084649,0.050870,-0.011415,...,0.000000,-0.001329,0.000000,0.000000,0.001331,-0.000665,-0.001328,-0.001326,-0.000663,-0.000663
1,-0.001229,0.018896,0.068262,0.124534,-0.350429,-0.026531,-0.398792,-0.084649,0.050870,-0.011415,...,-0.001995,0.000000,-0.001329,0.000000,0.000000,0.001331,-0.000665,-0.001328,-0.001326,-0.000663
2,-0.001231,0.018896,0.068262,0.124534,-0.350429,-0.026531,-0.398792,-0.084649,0.050870,-0.011415,...,-0.001332,-0.001995,0.000000,-0.001329,0.000000,0.000000,0.001331,-0.000665,-0.001328,-0.001326
3,-0.002465,0.018896,0.068262,0.124534,-0.350429,-0.026531,-0.398792,-0.084649,0.050870,-0.011415,...,-0.001334,-0.001332,-0.001995,0.000000,-0.001329,0.000000,0.000000,0.001331,-0.000665,-0.001328
4,-0.000618,0.018896,0.068262,0.124534,-0.350429,-0.026531,-0.398792,-0.084649,0.050870,-0.011415,...,0.000668,-0.001334,-0.001332,-0.001995,0.000000,-0.001329,0.000000,0.000000,0.001331,-0.000665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5115,0.000000,-0.013343,0.018896,0.068262,0.124534,-0.350429,-0.026531,-0.398792,-0.084649,0.050870,...,0.000000,0.000000,0.000605,-0.001209,-0.001208,-0.000603,0.000604,-0.000603,-0.001205,0.000000
5116,0.003541,-0.013343,0.018896,0.068262,0.124534,-0.350429,-0.026531,-0.398792,-0.084649,0.050870,...,0.000000,0.000000,0.000000,0.000605,-0.001209,-0.001208,-0.000603,0.000604,-0.000603,-0.001205
5117,0.002117,-0.013343,0.018896,0.068262,0.124534,-0.350429,-0.026531,-0.398792,-0.084649,0.050870,...,0.000605,0.000000,0.000000,0.000000,0.000605,-0.001209,-0.001208,-0.000603,0.000604,-0.000603
5118,0.000704,-0.013343,0.018896,0.068262,0.124534,-0.350429,-0.026531,-0.398792,-0.084649,0.050870,...,0.000604,0.000605,0.000000,0.000000,0.000000,0.000605,-0.001209,-0.001208,-0.000603,0.000604


In [ ]:
len(df)

In [15]:
515444/10

51544.4

In [17]:
1024*50

51200

In [16]:
import pickle
with open('TQQQ_processed.pkl','wb') as f:
    pickle.dump(df, f)

In [55]:
### some hyperparameters
MIN_DELTA = 0.001
MIN_LR = 0.0001
HIDDEN_SIZE = 64
NUM_LAYERS = 2
LEARNING_RATE = 0.01
NUM_EPOCHS = 50



In [56]:
### define data loader
class LargeDataset(Dataset):
    def __init__(self, data_dir, stock_code, block_index_list):
        self.stock_code = stock_code
        self.data_dir = data_dir
        self.file_list = [i for i in sorted(os.listdir(data_dir)) if stock_code in i and int(i.split('.pkl')[0].split('_')[-1]) in block_index_list]  # List all files in the data directory

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_path = os.path.join(self.data_dir, self.file_list[idx])
        with open(file_path, 'rb') as f:
            data = pickle.load(f)
        x, y = np.expand_dims(data.iloc[:,1:].values, -1), data.iloc[:,0].values  # Assuming the data is stored as dictionaries with 'x' and 'y' keys
        return x, y

### define data loader
class subDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        x, y = self.x[idx,:], self.y[idx]  # Assuming the data is stored as dictionaries with 'x' and 'y' keys
        return x, y
    
class myloader():
    def __init__(self, data_dir, stock_code, block_index_list):
        self.stock_code = stock_code
        self.block_index_list = block_index_list
        self.overall_train_dataset = LargeDataset(data_dir,stock_code,block_index_list)
        self.current_sub_dataset = None
    
    def get_data(self):
        for block_idx in range(len(self.block_index_list)):
            X_block, y_block = self.overall_train_dataset[block_idx]
            sub_dataloader = DataLoader(
                    subDataset(X_block, y_block),
                    batch_size=1024,
                    shuffle=False
                )
            for x,y in sub_dataloader:
                yield x,y

                    

In [57]:
### define some utils
class EarlyStopping():
    """
    Early stopping to stop the training when the loss does not improve after patience epochs.
    """
    def __init__(self, patience=10, min_delta=MIN_DELTA):
        """
        :param patience: how many epochs to wait before stopping when loss is not improving
        :param min_delta: minimum difference between new loss and old loss for
               new loss to be considered as an improvement
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
    def __call__(self, val_loss):
        if self.best_loss == None:
            self.best_loss = val_loss
        elif self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            # reset counter if validation loss improves
            self.counter = 0
        elif self.best_loss - val_loss < self.min_delta:
            self.counter += 1
            # print(f"INFO: Early stopping counter {self.counter} of {self.patience}")
            if self.counter >= self.patience:
                # print('INFO: Early stopping')
                self.early_stop = True
                
                
class LRScheduler():
    """
    Learning rate scheduler. If the validation loss does not decrease for the 
    given number of `patience` epochs, then the learning rate will decrease by
    by given `factor`.
    """
    def __init__(self, optimizer, patience=3, min_lr=MIN_LR, factor=0.1):
        """
        new_lr = old_lr * factor
        :param optimizer: the optimizer we are using
        :param patience: how many epochs to wait before updating the lr
        :param min_lr: least lr value to reduce to while updating
        :param factor: factor by which the lr should be updated
        """
        self.optimizer = optimizer
        self.patience = patience
        self.min_lr = min_lr
        self.factor = factor
        self.lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau( 
                self.optimizer,
                mode='min',
                patience=self.patience,
                factor=self.factor,
                min_lr=self.min_lr,
                verbose=True
            )
    def __call__(self, val_loss):
        # take one step of the learning rate scheduler while providing the validation loss as the argument
        self.lr_scheduler.step(val_loss)
        
        

In [62]:
#### define the model
class LSTM_model(nn.Module):
    def __init__(self, hidden_size, num_layers):
        super(LSTM_model, self).__init__()
        self.hidden_size = hidden_size # number of features in hidden state
        self.num_layers = num_layers #number of lstm layers
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        self.conv1 = nn.Conv1d(1, 8, kernel_size=64) # in_channels, out_channels, kernel_size
        self.pool1 = nn.MaxPool1d(kernel_size=4)
        self.conv2 = nn.Conv1d(8, 16,kernel_size=32)
        self.pool2 = nn.MaxPool1d(kernel_size=4)        
        self.conv3 = nn.Conv1d(16, 32,kernel_size=16)
        self.pool3 = nn.MaxPool1d(kernel_size=4)
        
        
        self.lstm_month = nn.LSTM(input_size=15, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True, bidirectional=False) #lstm
        self.lstm_week = nn.LSTM(input_size=15, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True, bidirectional=False) #lstm
        self.lstm_day = nn.LSTM(input_size=30, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True, bidirectional=False) #lstm
        self.lstm_minute = nn.LSTM(input_size=84, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True, bidirectional=False) #lstm

        self.fc_1 =  nn.Linear(266,256) #fully connected 1
        self.fc_2 =  nn.Linear(256,32) #fully connected 2
        self.fc_3 =  nn.Linear(32,1) #fully connected 3

        torch.nn.init.xavier_uniform_(self.fc_1.weight)
        torch.nn.init.xavier_uniform_(self.fc_2.weight)
        torch.nn.init.xavier_uniform_(self.fc_3.weight)
        
    def forward(self,x):
        ##### LSTM for month data
        month_data = x[:,:,0:15]
        h_0_month = Variable(torch.zeros(self.num_layers, month_data.size(0), self.hidden_size)).to(self.device)
        c_0_month = Variable(torch.zeros(self.num_layers, month_data.size(0), self.hidden_size)).to(self.device)
        output_month, (hn_month, cn_month) = self.lstm_month(month_data, (h_0_month, c_0_month)) # hn.shape = 1, N, 64
        hn_month = hn_month[hn_month.shape[0]-1:]
        hn_month = hn_month.view(-1, self.hidden_size) # hn.shape = N, 64
        
        ##### LSTM for week data
        week_data = x[:,:,15:30]
        h_0_week = Variable(torch.zeros(self.num_layers, week_data.size(0), self.hidden_size)).to(self.device)
        c_0_week = Variable(torch.zeros(self.num_layers, week_data.size(0), self.hidden_size)).to(self.device)
        output_week, (hn_week, cn_week) = self.lstm_month(week_data, (h_0_week, c_0_week)) # hn.shape = 1, N, 64
        hn_week = hn_week[hn_week.shape[0]-1:]
        hn_week = hn_month.view(-1, self.hidden_size) # hn.shape = N, 64
        
        ##### LSTM for day data
        day_data = x[:,:,30:60]
        h_0_day = Variable(torch.zeros(self.num_layers, day_data.size(0), self.hidden_size)).to(self.device)
        c_0_day = Variable(torch.zeros(self.num_layers, day_data.size(0), self.hidden_size)).to(self.device)
        output_day, (hn_day, cn_day) = self.lstm_day(day_data, (h_0_day, c_0_day)) # hn.shape = 1, N, 64
        hn_day = hn_day[hn_day.shape[0]-1:]
        hn_day = hn_day.view(-1, self.hidden_size) # hn.shape = N, 64
        
        
        ##### convolution for minute data
        # print(x.shape) # N, 1, 24
        out  = self.conv1(x[:,:,60:]) # default activation is None; out.shape: N, 8, 24
        out = self.pool1(out) # out.shape: N, 8, 12
        
        out = self.conv2(out) # out.shape: N, 16, 12
        out = self.pool2(out) # out.shape: N, 16, 6
        
        out = self.conv3(out) # out.shape: N, 32, 6
        out = self.pool3(out) # out.shape: N, 32, 3
        
        h_0 = Variable(torch.zeros(self.num_layers, out.size(0), self.hidden_size)).to(self.device)
        c_0 = Variable(torch.zeros(self.num_layers, out.size(0), self.hidden_size)).to(self.device)

        output, (hn, cn) = self.lstm_minute(out, (h_0, c_0)) # hn.shape = 1, N, 64
        hn = hn[hn.shape[0]-1]
        hn = hn.view(-1, self.hidden_size) # hn.shape = N, 64
        
        input_to_fcl = torch.cat((hn_month, hn_week, hn_day, hn, x[:,0,-10:]), axis=1)
        out = self.fc_1(input_to_fcl) # Dense, out.shape = N, 256
        out = self.fc_2(out) # out.shape = N, 32
        out = self.fc_3(out) # out.shape = N, 12
        return out
    

In [63]:
#### load data
train_loader = myloader('./training_data/','TQQQ',list(range(0,65)))
val_loader = myloader('./training_data/','TQQQ',list(range(65,75)))



In [64]:
#### define the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
lstm_model = LSTM_model(HIDDEN_SIZE, NUM_LAYERS)
lstm_model = lstm_model.to(device).float()
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=LEARNING_RATE)
lr_scheduler = LRScheduler(optimizer)
early_stopping = EarlyStopping()
        

In [65]:
##### start training
val_losses = []
train_losses = []
min_val_loss = np.inf


# # logging
# wandb.init(
#     project="TQQQ_prediciton_pure_hist_price",
#     config={
#     "learning_rate": LEARNING_RATE,
#     "architecture": "LSTM",
#     "dataset": "TQQQ",
#     "epochs": NUM_EPOCHS,
#     }
# )


for epoch in range(NUM_EPOCHS):
    
    ## train
    lstm_model.train()
    for inputs, targets in tqdm(train_loader.get_data(),total=len(train_loader.block_index_list)*5):
        inputs = inputs.float()
        targets = targets.float()
        inputs = inputs.permute([0,2,1])
        inputs = inputs.to(device)
        targets = targets.to(device)
        
        ### train, pred
        preds = lstm_model(inputs)
        loss = criterion(preds, targets)
        
        ### back prop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            train_losses.append(loss.item())
    
    ### eval
    r2_list = []
    spearmanr_list = []
    lstm_model.eval()
    for inputs, targets in val_loader.get_data():
        inputs = inputs.float()
        targets = targets.float()
        inputs = inputs.permute([0,2,1])
        inputs = inputs.to(device)
        preds = lstm_model(inputs)
        val_loss = criterion(preds, targets)

        val_losses.append(val_loss.item())
        
        ### r2 and spearman_r
        r2 = r2_score(targets.detach().numpy().flatten(),
                 preds.detach().numpy().flatten()
                )
        spearman_r = spearmanr(targets.detach().numpy().flatten(),
                 preds.detach().numpy().flatten()
                )[0]
        r2_list.append(r2)
        spearmanr_list.append(spearman_r)

    with torch.no_grad():
        val_epoch_loss = np.mean(val_losses)
        train_epoch_loss = np.mean(train_losses)
        if epoch%1 == 0 or epoch==(NUM_EPOCHS-1):
            print("Epoch: {}/{}...".format(epoch, NUM_EPOCHS),
                  "Loss: {:.6f}...".format(train_epoch_loss),
                  "Val Loss: {:.6f}".format(val_epoch_loss))

        if val_epoch_loss < min_val_loss:
            min_val_loss = val_epoch_loss
#             best_hidden_size = hidden_size
            best_model = lstm_model

        lr_scheduler(val_epoch_loss)
        early_stopping(val_epoch_loss)
        if early_stopping.early_stop:
            print("Early stopping after epoch: {}/{}...".format(epoch, NUM_EPOCHS),
                  "Loss: {:.6f}...".format(np.mean(train_losses)),
                  "Val Loss: {:.6f}".format(val_epoch_loss))            
            break
            
    for param_group in lr_scheduler.optimizer.param_groups:
        this_lr = param_group['lr']
    wandb.log({"train_loss": train_epoch_loss, "val_loss": val_epoch_loss, 
               'val_r2':np.nanmean(r2_list), 'val_spearmanr':np.nanmean(spearmanr_list), 'lr':this_lr})

wandb.finish()


  1%|          | 2/325 [00:19<52:23,  9.73s/it]


KeyboardInterrupt: 

In [21]:
with torch.no_grad():
    for inputs, targets in tqdm(train_loader.get_data(),total=len(train_loader.block_index_list)*5):
        inputs = inputs.float()
        targets = targets.float()
        inputs = inputs.permute([0,2,1])
        inputs = inputs.to(device)
        targets = targets.to(device)
        
        ### train, pred
        preds = lstm_model(inputs)
        loss = criterion(preds, targets)
        
        ### back prop
        break

  0%|          | 0/325 [00:02<?, ?it/s]


In [25]:
targets.shape

torch.Size([1024])

In [24]:
len(preds.flatten())

2048

In [43]:
spearman_r

nan

In [44]:
spearmanr(targets.detach().numpy(),
         preds.detach().numpy().flatten()
        )

SignificanceResult(statistic=nan, pvalue=nan)

In [27]:
train_losses

[0.036112137138843536,
 1427162.75,
 12306202.0,
 5488707072.0,
 25637078.0,
 17547354.0,
 12477466.0,
 9092305.0,
 6736295.5,
 5048871.5,
 3815373.0,
 2900171.0,
 2213623.75,
 1694489.5,
 1299653.0,
 997935.5625,
 766811.9375,
 589319.125,
 452861.4375,
 347869.0,
 267043.90625,
 204845.171875,
 156990.25,
 120195.78125,
 91918.421875,
 70216.9375,
 53570.51953125,
 40816.1640625,
 31059.31640625,
 23603.078125,
 17912.25390625,
 13572.0078125,
 10270.884765625,
 7760.15673828125,
 5854.61865234375,
 4410.3740234375,
 3318.45654296875,
 2492.680419921875,
 1869.0374755859375,
 1399.6297607421875,
 1046.38134765625,
 781.0643310546875,
 582.0787353515625,
 433.144287109375,
 321.8984680175781,
 238.70437622070312,
 176.73374938964844,
 130.71511840820312,
 96.43936157226562,
 71.23314666748047,
 52.33595275878906,
 38.38673400878906,
 28.20049285888672,
 20.608150482177734,
 15.104609489440918,
 10.995914459228516,
 8.032283782958984,
 5.81132173538208,
 4.229701042175293,
 3.062998294

In [66]:
transformer_model = nn.Transformer(nhead=16, num_encoder_layers=12)
src = torch.rand((10, 32, 512))
tgt = torch.rand((20, 32, 512))
out = transformer_model(src, tgt)

In [ ]:
out.shape